<a href="https://colab.research.google.com/github/masx200/langchain_1_0_models_complete_guide_ipynb/blob/main/langchain_1_0_models_complete_guide_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain 1.0 模型声明与使用完全指南

本notebook全面介绍LangChain 1.0中模型的声明和使用方法，突出其灵活性和多样性。

## 目录
1. [环境准备](#1-环境准备)
2. [基础：固定模型声明](#2-基础固定模型声明)
3. [灵活：无默认模型的可配置模型](#3-灵活无默认模型的可配置模型)
4. [混合：有默认模型的可配置模型](#4-混合有默认模型的可配置模型)
5. [高级：绑定工具的可配置模型](#5-高级绑定工具的可配置模型)

## 1. 环境准备

安装必要的依赖包：

In [ ]:
!pip install langchain langchain-openai langchain-anthropic langchain-google-vertexai

In [ ]:
# 安装核心包和提供商集成
# !pip install langchain langchain-openai langchain-anthropic langchain-google-vertexai

import os
from google.colab import userdata

# 设置API密钥
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 如果使用Anthropic
# if "ANTHROPIC_API_KEY" not in os.environ:
#     os.environ["ANTHROPIC_API_KEY"] = getpass("请输入Anthropic API Key: ")

# 如果使用Google Vertex AI
# if "GOOGLE_APPLICATION_CREDENTIALS" not in os.environ:
#     os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/credentials.json"

## 2. 基础：固定模型声明

最直接的使用方式：声明固定的模型并调用。

### 2.1 声明不同提供商的模型

使用 `provider:model` 格式明确指定提供商和模型。

In [ ]:
from langchain.chat_models import init_chat_model

# 声明OpenAI模型
o3_mini = init_chat_model("openai:o3-mini")

# 声明Anthropic模型
claude_sonnet = init_chat_model("anthropic:claude-sonnet-4-5-20250929", temperature=0)

# 声明Google Vertex AI模型
# gemini_2_5_flash = init_chat_model("google_vertexai:gemini-2.5-flash", temperature=0)

print("模型声明完成")

模型声明完成


In [ ]:
print(o3_mini.dict())

{'model_name': 'o3-mini', 'model': 'o3-mini', 'stream': False, '_type': 'openai-chat'}


### 2.2 调用不同模型

每个模型独立调用，返回各自的结果。

In [ ]:
# 调用OpenAI模型
response_o3 = o3_mini.invoke("what's your name")
print("O3-mini:", response_o3.content)

# 调用Anthropic模型
response_claude = claude_sonnet.invoke("what's your name")
print("\nClaude Sonnet:", response_claude.content)

# 调用Google模型
# response_gemini = gemini_2_5_flash.invoke("what's your name")
# print("\nGemini 2.5 Flash:", response_gemini.content)

O3-mini: I'm ChatGPT—a language model created by OpenAI. How can I help you today?


TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [ ]:
from langchain_openai import ChatOpenAI

o3_mini_model = ChatOpenAI(model="o3-mini")

In [ ]:
o3_mini_model.invoke("Hi")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 7, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_d83b50479d', 'id': 'chatcmpl-CaqZe4O8JFI0hZiuWxysTvoxHfCJi', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--51865903-63ab-4e26-9e4f-fb8bf5452549-0', usage_metadata={'input_tokens': 7, 'output_tokens': 150, 'total_tokens': 157, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

### 2.3 自动推断提供商

LangChain可以根据模型名称前缀自动推断提供商。

In [ ]:
from langchain.chat_models import init_chat_model

# 不需要显式指定提供商，系统自动识别
gpt_4o = init_chat_model("gpt-4o-mini", temperature=0)  # 自动识别为OpenAI
# claude = init_chat_model("claude-sonnet-4-5-20250929", temperature=0)  # 自动识别为Anthropic

response = gpt_4o.invoke("what's your name")
print("GPT-4o-mini:", response.content)

GPT-4o-mini: I’m called ChatGPT. How can I assist you today?


### 2.4 设置模型参数

在初始化时设置temperature、max_tokens等参数。

In [ ]:
from langchain.chat_models import init_chat_model

# 设置不同的参数
model_deterministic = init_chat_model("gpt-4o-mini", temperature=0)  # 确定性输出
model_creative = init_chat_model("gpt-4o-mini", temperature=1)  # 创造性输出
model_limited = init_chat_model("gpt-4o-mini", temperature=0, max_tokens=50)  # 限制输出长度

prompt = "给我讲一个笑话"

print("Temperature=0:")
print(model_deterministic.invoke(prompt).content)

print("\nTemperature=1:")
print(model_creative.invoke(prompt).content)

print("\nMax tokens=50:")
print(model_limited.invoke(prompt).content)

Temperature=0:
当然可以！这是一个简单的笑话：

有一天，一只鸡走进了图书馆。它走到图书馆员那里，咕咕叫了几声。图书馆员有些困惑，但还是给了它一本书。鸡拿着书走了出去。

第二天，鸡又来了，还是咕咕叫。图书馆员又给了它一本书。鸡又走了。

第三天，鸡再次来到图书馆，这次它叫得更急了。图书馆员决定跟着它，看看它到底在做什么。

鸡走到一个池塘边，把书递给了一只青蛙。青蛙看了看书，摇了摇头，说：“呱，不是这个，呱，不是这个！”

希望这个笑话能让你笑一笑！

Temperature=1:
当然可以！这里有一个笑话：

为什么海洋总是那么蓝？

因为鱼在游的时候总是“藍藍藍”！ 

希望这个笑话能让你开心！

Max tokens=50:
当然可以！这是一个简单的笑话：

有一天，一只鸡走进了图书馆。它走到图书馆员那里，咕咕地说：“书，书，书！”  
图书馆员有


## 3. 灵活：无默认模型的可配置模型

声明一个可配置模型，不指定默认模型，在运行时动态指定使用哪个模型。

### 3.1 声明完全可配置的模型

不指定模型名称，创建一个可以在运行时配置的模型实例。

In [ ]:
from langchain.chat_models import init_chat_model

# 不指定model参数，意味着必须在运行时通过config指定
# configurable_fields会自动设置为"any"
configurable_model = init_chat_model(temperature=0)

print("可配置模型创建完成（无默认模型）")

可配置模型创建完成（无默认模型）


### 3.2 运行时指定不同模型

通过 `config` 参数在每次调用时指定使用的模型。

In [ ]:
# 使用GPT-4o生成响应
response_gpt = configurable_model.invoke(
    "what's your name",
    config={"configurable": {"model": "gpt-4o"}}
)
print("使用 GPT-4o:")
print(response_gpt.content)

# 使用Claude Sonnet生成响应
# response_claude = configurable_model.invoke(
#     "what's your name",
#     config={"configurable": {"model": "claude-sonnet-4-5-20250929"}}
# )
# print("\n使用 Claude Sonnet:")
# print(response_claude.content)

使用 GPT-4o:
I’m called ChatGPT. How can I assist you today?


### 3.3 运行时指定模型和参数

除了模型，还可以在运行时指定其他参数。

In [ ]:
from langchain.chat_models import init_chat_model

# 创建可配置模型，设置默认temperature
configurable_model = init_chat_model(temperature=0)

# 运行时同时指定模型和参数
response = configurable_model.invoke(
    "讲一个笑话",
    config={
        "configurable": {
            "model": "gpt-4o-mini",
            "temperature": 0.9,  # 覆盖默认的temperature=0
            "max_tokens": 100
        }
    }
)

print(response.content)

有一天，小明去买水果。他看到一个摊位上有很多苹果，便问老板：“这些苹果多少钱一个？”

老板回答：“一个五块，三个十块。”

小明想了想，问：“那我买两个，能不能给我打个折？”

老板笑着说：“当然可以，你可以买一个五块，另一个也五块，这样总共十块，没打折！”

小明无奈地说：“那我还是买三个吧，反正十块也是十块！”


### 3.4 使用场景：动态模型选择

适用于需要根据不同条件选择不同模型的场景。

In [ ]:
from langchain.chat_models import init_chat_model

# 创建可配置模型
flexible_model = init_chat_model(temperature=0)

def ask_model(question: str, use_advanced: bool = False):
    """根据需求选择模型"""
    model_name = "gpt-4o" if use_advanced else "gpt-4o-mini"

    response = flexible_model.invoke(
        question,
        config={"configurable": {"model": model_name}}
    )
    return response.content

# 简单问题用小模型
print("简单问题 (gpt-4o-mini):")
print(ask_model("1+1=?", use_advanced=False))

# 复杂问题用大模型
print("\n复杂问题 (gpt-4o):")
print(ask_model("解释量子纠缠", use_advanced=True)[:100], "...")

简单问题 (gpt-4o-mini):
1 + 1 = 2.

复杂问题 (gpt-4o):
量子纠缠是量子力学中的一种现象，其中两个或多个粒子在量子状态上相互关联，以至于无论它们相距多远，对其中一个粒子的测量结果会立即影响另一个粒子的状态。这种现象最初由爱因斯坦、波多尔斯基和罗森在1935年 ...


## 4. 混合：有默认模型的可配置模型

声明一个带默认模型的可配置模型，可以在运行时选择性地覆盖默认设置。

### 4.1 声明带默认值的可配置模型

指定默认模型和参数，同时允许运行时覆盖。

In [ ]:
from langchain.chat_models import init_chat_model

# 指定默认模型和参数，同时允许运行时配置
configurable_model_with_default = init_chat_model(
    "openai:gpt-4o",
    configurable_fields="any",  # 允许运行时配置所有字段
    temperature=0,
)

print("带默认值的可配置模型创建完成")
print("默认: gpt-4o, temperature=0")

带默认值的可配置模型创建完成
默认: gpt-4o, temperature=0


### 4.2 使用默认配置

不传config参数时，使用初始化时的默认设置。

In [ ]:
# 使用默认配置：gpt-4o + temperature=0
response = configurable_model_with_default.invoke("what's your name")

print("使用默认配置 (gpt-4o, temperature=0):")
print(response.content)

使用默认配置 (gpt-4o, temperature=0):
I’m called ChatGPT. How can I assist you today?


### 4.3 运行时覆盖配置

传入config参数覆盖默认的模型和参数。

In [ ]:
# 覆盖默认配置：使用Claude Sonnet + temperature=0.6
# response = configurable_model_with_default.invoke(
#     "what's your name",
#     config={
#         "configurable": {
#             "model": "anthropic:claude-sonnet-4-5-20250929",
#             "temperature": 0.6,
#         }
#     },
# )

# print("覆盖配置 (claude-sonnet, temperature=0.6):")
# print(response.content)

print("演示代码（需要Anthropic API密钥）")

### 4.4 使用config_prefix管理配置

使用配置前缀为多个模型创建独立的命名空间。

In [ ]:
from langchain.chat_models import init_chat_model

# 使用config_prefix创建带命名空间的配置
model_with_prefix = init_chat_model(
    "openai:gpt-4o",
    configurable_fields="any",
    config_prefix="foo",  # 配置前缀
    temperature=0,
)

# 使用默认配置
response1 = model_with_prefix.invoke("what's your name")
print("默认配置:")
print(response1.content)

# 使用前缀覆盖配置
# 注意：配置键需要加上前缀 "foo_"
# response2 = model_with_prefix.invoke(
#     "what's your name",
#     config={
#         "configurable": {
#             "foo_model": "anthropic:claude-sonnet-4-5-20250929",  # 前缀_model
#             "foo_temperature": 0.6,  # 前缀_temperature
#         }
#     },
# )

# print("\n覆盖配置 (使用前缀):")
# print(response2.content)

print("\n配置前缀用于管理多个可配置模型")

### 4.5 多模型协作示例

使用config_prefix管理多个模型的配置。

In [ ]:
from langchain.chat_models import init_chat_model

# 总结器模型
summarizer = init_chat_model(
    "gpt-4o-mini",
    configurable_fields="any",
    config_prefix="summarizer",
    temperature=0,
)

# 翻译器模型
translator = init_chat_model(
    "gpt-4o-mini",
    configurable_fields="any",
    config_prefix="translator",
    temperature=0,
)

# 统一配置两个模型
unified_config = {
    "configurable": {
        "summarizer_model": "gpt-4o-mini",
        "summarizer_temperature": 0,
        "translator_model": "gpt-4o",
        "translator_temperature": 0.3,
    }
}

text = "人工智能正在改变世界。它在医疗、教育、交通等领域都有广泛应用。"

# 总结
summary = summarizer.invoke(
    f"用一句话总结：{text}",
    config=unified_config
)
print("总结:", summary.content)

# 翻译
translation = translator.invoke(
    f"翻译成英文：{summary.content}",
    config=unified_config
)
print("\n翻译:", translation.content)

总结: 人工智能正在通过在医疗、教育和交通等领域的广泛应用，深刻改变我们的世界。

翻译: Artificial intelligence is profoundly transforming our world through its widespread applications in fields such as healthcare, education, and transportation.


## 总结

本notebook介绍了LangChain 1.0中模型声明和使用的核心模式：

### 三种主要模式

#### 1. 固定模型（Fixed Model）
```python
model = init_chat_model("openai:gpt-4o", temperature=0)
model.invoke("question")
```
- **特点**: 模型和参数在初始化时固定
- **优势**: 简单直接，配置明确
- **适用**: 单一模型应用，生产环境

#### 2. 无默认模型的可配置模型（Fully Configurable）
```python
model = init_chat_model(temperature=0)  # 不指定模型
model.invoke("question", config={"configurable": {"model": "gpt-4o"}})
```
- **特点**: 必须在运行时指定模型
- **优势**: 最大灵活性，完全动态
- **适用**: 多模型切换，A/B测试，用户选择模型

#### 3. 有默认模型的可配置模型（Configurable with Default）
```python
model = init_chat_model("gpt-4o", configurable_fields="any", temperature=0)
model.invoke("question")  # 使用默认
model.invoke("question", config={"configurable": {"model": "claude-sonnet"}})  # 覆盖
```
- **特点**: 有默认值，可选择性覆盖
- **优势**: 兼顾便利性和灵活性
- **适用**: 大部分场景用默认，特殊情况切换

### 核心参数

| 参数 | 说明 | 示例 |
|------|------|------|
| `model` | 模型标识符 | `"openai:gpt-4o"`, `"claude-sonnet"` |
| `temperature` | 随机性控制 (0-1) | `0` (确定), `1` (创造) |
| `configurable_fields` | 可配置字段 | `"any"`, `("model", "temperature")` |
| `config_prefix` | 配置前缀 | `"foo"` → 配置键为 `"foo_model"` |
| `max_tokens` | 最大输出长度 | `100`, `1000` |

### 高级特性

- **工具绑定**: 使用 `.bind_tools()` 为模型添加函数调用能力
- **配置前缀**: 使用 `config_prefix` 管理多个可配置模型
- **运行时覆盖**: 通过 `config` 参数灵活调整任何参数

### 最佳实践

1. **开发阶段**: 使用可配置模型快速实验不同模型
2. **生产环境**: 根据稳定性需求选择固定或可配置模式
3. **多模型应用**: 使用 `config_prefix` 管理多个模型的配置
4. **成本优化**: 默认使用小模型，复杂任务时切换大模型
5. **工具调用**: 选择支持工具调用的模型（GPT-4o, Claude等）

LangChain 1.0的设计理念：**统一接口 + 灵活配置**，让你可以轻松切换15+个提供商的模型，同时保持代码简洁。